# 3. Get GSE tasks for all airports

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime
from datetime import time, timedelta
import get_GSE_tasks as TASK
import matplotlib.pyplot as plt

In [2]:
def fix_time_format(date_str): # a function to fix "24:00" issue
    if "24:00" in date_str:
        date_part = date_str.split()[0]  # Extract date
        new_date = datetime.strptime(date_part, "%m/%d/%Y") + timedelta(days=1)  # Add a day
        return new_date.strftime("%m/%d/%Y 00:00")  # Format correctly
    return date_str

In [3]:
## Define a function to run each airport in parallel

def single_airport_add_task(file_path):
    all_flight_data = pd.read_csv(file_path)
    all_flight_data['Arrival_time'] = all_flight_data['Date (MM/DD/YYYY)'] + ' ' + all_flight_data['Actual Arrival Time']
    all_flight_data['Arrival_time'] = all_flight_data['Arrival_time'].apply(fix_time_format)
    all_flight_data['Arrival_time'] = pd.to_datetime(all_flight_data['Arrival_time'])
    all_flight_data['aircraft_type'] = all_flight_data['Carrier Code'].apply(lambda x: 'wide' if x == 'III' else 'narrow')
    GSE_tasks = TASK.get_gse_schedule(all_flight_data)
    airport = os.path.basename(file_path).split('_')[0]
    GSE_tasks.to_csv('./all_flight_GSE_tasks/{}_GSE_tasks.csv'.format(airport))    

In [4]:
# Parallelizing using Pool.map()
import multiprocessing as mp

## All raw data
folder_path = Path('./all_flight_arrival_data_New')
all_files_New = folder_path.glob('*.csv')

pool = mp.Pool(mp.cpu_count())
 
results = pool.map(single_airport_add_task, list(all_files_New))
 
pool.close()